In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch.nn import BatchNorm1d
from sklearn.metrics import confusion_matrix, classification_report
from torch.optim.lr_scheduler import StepLR
import torch
torch.cuda.empty_cache()

In [40]:
data = pd.read_csv('C:\\big18\\final\with\\almost\\test_data\\000660_2체결강도Vector.csv')

In [43]:
data = data.drop('18', axis=1)

In [44]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,29.0023,110.5294,35.2941,0.0000,-23.6128,-23.5571,41.1281,-5.8997,29.4811,102.44,13.77,1.10,-24.22,-6.77,-12.96,46.09,-41.80,49.52
1,110.5294,35.2941,0.0000,-23.6128,-23.5571,41.1281,-5.8997,29.4811,29.5683,13.77,1.10,-24.22,-6.77,-12.96,46.09,-41.80,49.52,5.56
2,35.2941,0.0000,-23.6128,-23.5571,41.1281,-5.8997,29.4811,29.5683,-35.5872,1.10,-24.22,-6.77,-12.96,46.09,-41.80,49.52,5.56,-41.44
3,0.0000,-23.6128,-23.5571,41.1281,-5.8997,29.4811,29.5683,-35.5872,5.9102,-24.22,-6.77,-12.96,46.09,-41.80,49.52,5.56,-41.44,-5.59
4,-23.6128,-23.5571,41.1281,-5.8997,29.4811,29.5683,-35.5872,5.9102,17.7410,-6.77,-12.96,46.09,-41.80,49.52,5.56,-41.44,-5.59,8.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,0.0000,5.7438,-5.7471,-5.7438,11.4811,0.0000,0.0000,-5.7471,5.7438,-1.13,6.45,-8.38,5.48,2.38,-4.05,-0.81,5.86,77.01
365,5.7438,-5.7471,-5.7438,11.4811,0.0000,0.0000,-5.7471,5.7438,0.0000,6.45,-8.38,5.48,2.38,-4.05,-0.81,5.86,77.01,-84.71
366,-5.7471,-5.7438,11.4811,0.0000,0.0000,-5.7471,5.7438,0.0000,-11.4943,-8.38,5.48,2.38,-4.05,-0.81,5.86,77.01,-84.71,-10.67
367,-5.7438,11.4811,0.0000,0.0000,-5.7471,5.7438,0.0000,-11.4943,0.0000,5.48,2.38,-4.05,-0.81,5.86,77.01,-84.71,-10.67,11.17


In [56]:


#      복잡 모델
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.bn1 = BatchNorm1d(16)
        self.conv2 = GCNConv(16, 32)
        self.bn2 = BatchNorm1d(32)
        self.conv3 = GCNConv(32, 64)
        self.bn3 = BatchNorm1d(64)
        self.conv4 = GCNConv(64, num_classes)
        # self.bn4 = BatchNorm1d(128)
        # self.conv5 = GCNConv(128, num_classes)
        # self.bn5 = BatchNorm1d(256)
        # self.conv6 = GCNConv(256, num_classes)
        # self.bn6 = BatchNorm1d(512)
        # self.conv7 = GCNConv(512, num_classes)
        self.dropout = torch.nn.Dropout(p=0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.gelu(x)
        # x = self.dropout(x)

        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.gelu(x)
        # x = self.dropout(x)

        x = self.conv3(x, edge_index)
        x = self.bn3(x)
        x = F.gelu(x)
        x = self.dropout(x)
        
        x = self.conv4(x, edge_index)
        # x = self.bn4(x)
        # x = F.gelu(x)
        # x = self.dropout(x)
        
        # x = self.conv5(x, edge_index)
        # x = self.bn5(x)
        # x = F.gelu(x)
        # x = self.dropout(x)
        
        # x = self.conv6(x, edge_index)
        # x = self.bn6(x)
        # x = F.gelu(x)
        # x = self.dropout(x)
        
        # x = self.conv7(x, edge_index)

        return F.log_softmax(x, dim=1)

답지list0 = []
답지list1 = []
답지list2 = []
임시 = []

for i in range(len(data) - 10):
    results_df = data[i : i + 10]
    # 모델 인스턴스 생성
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    real_data = results_df
    real_x = torch.tensor(real_data.values, dtype=torch.float)
    print(real_x)
    # 엣지 리스트 (Edge list)

    edge_index = []
    # num_rows = len(real_x)
    # for j in range(num_rows - 1):
    #     edge_index.append([j, j + 1])
    #     edge_index.append([j + 1, j])

    num_rows = len(real_x)
    for j in range(2, num_rows):
        edge_index.append([j , j - 1])
        edge_index.append([j , j - 2])


    real_edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()  # 빈 엣지 리스트 생성

    # real_edge_index = torch.tensor([[0],[0]], dtype=torch.long)
    real_data = Data(x=real_x, edge_index=real_edge_index)


    model = GCN(num_node_features=real_x.size(1), num_classes=3)
    model = model.to(device)
    real_data = real_data.to(device)

    # 저장된 모델 로드
    model.load_state_dict(torch.load('C:\\big18\\final\with\\almost\\gcn_model\\gcn변동률.pth'))
    model.eval()  # 모델 평가 모드로 전환

    # 예측 수행
    with torch.no_grad():
        real_out = model(real_data)
        real_pred = real_out.max(dim=1)[1].cpu().numpy()

    # 결과를 데이터프레임에 추가
    # real_close_data = pd.read_csv(f'C:/big18/dl-dev/dl-dev/project/변동률과 체결강도/{code}.csv', encoding='cp949')
    # real_close_data = real_close_data['종가']

    # real_pred = real_pred.astype(float)
    # real_pred = np.insert(real_pred, 0, [np.nan] * 12)

    # results_df2 = pd.DataFrame({
    #     # '종가': real_close_data,
    #     '예측 행동': real_pred
    # })

    # 결과 출력
    # print(results_df2)
    답지list0.append(real_pred[0])
    답지list1.append(real_pred[1])
    답지list2.append(real_pred[2])
    임시.append(real_pred)



    # 백테스팅 로직을 여기에 추가
    # 이전의 백테스팅 코드를 재사용
    # ...

    # 최종 자산 가치 계산 및 출력
    # ...


tensor([[ 29.0023, 110.5294,  35.2941,   0.0000, -23.6128, -23.5571,  41.1281,
          -5.8997,  29.4811, 102.4400,  13.7700,   1.1000, -24.2200,  -6.7700,
         -12.9600,  46.0900, -41.8000,  49.5200],
        [110.5294,  35.2941,   0.0000, -23.6128, -23.5571,  41.1281,  -5.8997,
          29.4811,  29.5683,  13.7700,   1.1000, -24.2200,  -6.7700, -12.9600,
          46.0900, -41.8000,  49.5200,   5.5600],
        [ 35.2941,   0.0000, -23.6128, -23.5571,  41.1281,  -5.8997,  29.4811,
          29.5683, -35.5872,   1.1000, -24.2200,  -6.7700, -12.9600,  46.0900,
         -41.8000,  49.5200,   5.5600, -41.4400],
        [  0.0000, -23.6128, -23.5571,  41.1281,  -5.8997,  29.4811,  29.5683,
         -35.5872,   5.9102, -24.2200,  -6.7700, -12.9600,  46.0900, -41.8000,
          49.5200,   5.5600, -41.4400,  -5.5900],
        [-23.6128, -23.5571,  41.1281,  -5.8997,  29.4811,  29.5683, -35.5872,
           5.9102,  17.7410,  -6.7700, -12.9600,  46.0900, -41.8000,  49.5200,
          

In [57]:
임시

[array([2, 2, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
 array([0, 0, 0, 2, 2, 2, 2, 2, 2, 2], dtype=int64),
 array([1, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64),
 array([2, 2, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
 array([0, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64),
 array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64),
 array([2, 2, 2, 1, 0, 0, 0, 0, 0, 0], dtype=int64),
 array([0, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64),
 array([2, 2, 2, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
 array([0, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64),
 array([1, 2, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
 array([2, 1, 0, 2, 2, 2, 2, 2, 2, 2], dtype=int64),
 array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
 array([0, 2, 0, 1, 1, 1, 1, 1, 2, 2], dtype=int64),
 array([2, 0, 0, 0, 2, 2, 2, 2, 2, 2], dtype=int64),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
 array([2, 0, 1, 2, 2, 2, 2, 2, 2, 2], dtype=int64),
 array([0, 0, 2, 2, 1, 0, 0, 0, 0, 0], dtype=int64),
 array([2, 2, 2, 1, 0, 0, 0, 0, 0, 0], dtype=i